In [172]:
import os
import torch

import prismtoolbox as ptb
import prismtoolbox.wsiemb as ptb_emb

import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from statannotations.Annotator import Annotator
from pathlib import Path

In [ ]:
df = pd.read_csv("../materials/dataset_lung_final_v2.csv")
df

# Daisy clustering analysis

In [441]:
output_dir = "../Results/LUNG/clustering/Daisy_analysis"
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [471]:
emb_type = "clam"

In [472]:
slides_to_analyse = pd.read_csv(f"../materials/slides_to_analyse_DAISY_analysis_LUNG.csv")
slide_names = slides_to_analyse["slide_name"].values
feats_dir = f"../Results/LUNG/clustering/feats_80_{emb_type}/"
feats_files = [os.path.join(feats_dir, slide_name+".pt") for slide_name in slide_names]
emb_processor = ptb_emb.EmbeddingProcessor(feats_files, slide_ids=slide_names, cmap="Set1")

In [ ]:
optimal_nb, scores = emb_processor.get_optimal_number_clusters("kmeans_mini_batch",
                                                                init="k-means++",
                                                                init_size = 3000,
                                                                batch_size=1000,
                                                                n_init=500,
                                                                max_no_improvement=100,
                                                                min_clusters=7,
                                                                max_clusters=12,
                                                                metric_name="davies_bouldin",
                                                                normalize=True,
                                                                random_state=1)
optimal_nb, scores

In [24]:
pd.DataFrame(scores).to_csv(os.path.join(output_dir, f"scores_{emb_type}.csv"), index=False)

In [405]:
emb_processor.create_cluster_model("kmeans_mini_batch",
                                    n_clusters=8,
                                    init="k-means++",
                                    init_size = 3000,
                                    batch_size=1000,
                                    n_init=500,
                                    max_no_improvement=100,
                                    random_state=1)

In [26]:
emb_processor.save_cluster_model(os.path.join(output_dir, f"cluster_model_{emb_type}.pkl"))

In [473]:
emb_processor.import_cluster_model(os.path.join(output_dir, f"cluster_model_{emb_type}.pkl"))

In [ ]:
cluster_percentages = [emb_processor.get_cluster_percentages_for_slide(slide_name.split(".")[0]) for slide_name in df["ihc_slide_id"].values]
labels = df.Confirmed_OR.map({1: "Responder", 0: "Non-responder"}).values
labels

In [475]:
df_plot = pd.DataFrame(cluster_percentages)
df_plot["Response to treatment"] = labels
df_plot.iloc[:,:-1] *= 100
df_plot.to_csv(os.path.join(output_dir, f"cluster_percentages_{emb_type}.csv"), index=False)
df_plot = df_plot.melt(id_vars='Response to treatment', var_name='Cluster', value_name='Percentage')
df_plot.to_csv(os.path.join(output_dir, f"cluster_percentages_{emb_type}_melted.csv"), index=False)

In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Cluster', y='Percentage', hue='Response to treatment', data=df_plot)
ax.set_title('Boxplot of Cluster Percentages Stratified  by Response to Treatment')
plt.savefig(os.path.join(output_dir, f"boxplot_{emb_type}.png"), dpi=300, bbox_inches="tight")

In [481]:
bbox_dir = "../Results/LUNG_bis/bbox_IHC_reg/"
coord_pretrained_dir = "../Results/LUNG/clustering/patches_80_overlap_0_annotations_IHC/"
coord_warped_dir = "../Results/LUNG/patches_80_overlap_0_annotations_IHC_warped/"
interpretable_feats_dir = "../Results/LUNG/feats_80_cell_stain_based_reg_warped_merged_IHC/"
embeddings_names = pd.read_csv("../Results/LUNG/feats_80_cell_stain_based_reg_warped_merged_IHC/embeddings_names.csv")["feature_names"].tolist()
selected_feats = [name for name in embeddings_names if "cells" in name or "DAB" in name]
slides_to_analyse = pd.read_csv(f"../materials/slides_to_analyse_New_cluster_analysis_LUNG.csv")
slide_names = slides_to_analyse["slide_name"].values
for slide_name_HE in slide_names:
    slide_name_IHC =  slide_name_HE.replace("B0-2", "B0-1")
    coord_pretrained = ptb.utils.data_utils.read_h5_file(os.path.join(coord_pretrained_dir, f"{slide_name_IHC}.h5"), "coords")[0]
    coord_warped = ptb.utils.data_utils.read_h5_file(os.path.join(coord_warped_dir, f"{slide_name_IHC}.h5"), "coords")[0]
    bbox = ptb.utils.data_utils.read_json_with_geopandas(os.path.join(bbox_dir, f"{slide_name_IHC}.geojson")).iloc[0].geometry
    indices = [np.where((coord_pretrained == (coord + bbox.bounds[:2]).astype("int64")).all(axis=1))[0].item() for coord in coord_warped]
    cluster_assign = emb_processor.get_cluster_assignments_for_slide(slide_name_IHC)[indices]
    interpretable_feats = torch.load(os.path.join(interpretable_feats_dir, f"{slide_name_HE}.pt"))
    selected_feats_idx = np.array([i for i, name in enumerate(embeddings_names) if name in selected_feats])
    features = pd.DataFrame(interpretable_feats[:, selected_feats_idx], columns=selected_feats)
    features['cluster'] = cluster_assign
features_to_plot = [name for name in features.columns if "DAB_avg" in name or "N_cells" in name] + ["cluster"]
features.loc[:, features_to_plot].to_csv(os.path.join(output_dir, f"features_cells_feats_{emb_type}.csv"), index=False)


# New clustering analysis

In [498]:
output_dir = "../Results/LUNG/clustering/New_cluster_analysis"
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [499]:
with_DAB = True
suffix = "DAB" if with_DAB else "only"

In [500]:
reg_HE2IHC = "../Results/LUNG/clustering/reg_HE2IHC/"
slides = []
error_HE2IHC = []
for slide in os.listdir(reg_HE2IHC):
    slides.append(slide)
    df_HE2IHC = pd.read_csv(os.path.join(reg_HE2IHC, slide, "data", f"{slide}_summary.csv"))
    error_HE2IHC.append(df_HE2IHC["non_rigid_rTRE"].dropna().values[0])

In [ ]:
df_reg_error = pd.DataFrame({"slide": slides, "error_HE2IHC": error_HE2IHC})
df_reg_error.sort_values(by="error_HE2IHC", ascending=False).head(10)

In [509]:
slides_to_analyse = pd.read_csv(f"../materials/slides_to_analyse_New_cluster_analysis_LUNG.csv")
slide_names = slides_to_analyse["slide_name"].values
feats_dir = "../Results/LUNG/clustering/feats_128_cell_stain_based_reg_warped_merged_HE_new/"
feats_files = [os.path.join(feats_dir, slide_name+".pt") for slide_name in slide_names]
emb_processor = ptb_emb.EmbeddingProcessor(feats_files, embeddings_names=os.path.join(feats_dir, "embeddings_names.csv"), slide_ids=slide_names, cmap="Set1")#

In [503]:
if with_DAB:
    selected_feats = [name for name in emb_processor.embeddings_names if "cells" in name or "DAB" in name]
else:
    selected_feats = [name for name in emb_processor.embeddings_names if "cells" in name]

In [ ]:
optimal_nb, scores = emb_processor.get_optimal_number_clusters("kmeans",
                                          selected_features=selected_feats,
                                          n_init=20,#20
                                          min_clusters=7,
                                          max_clusters=12,
                                          metric_name="calinski_harabasz",
                                          normalize=True,
                                          random_state=1)
optimal_nb, scores

In [166]:
pd.DataFrame(scores).to_csv(os.path.join(output_dir, f"scores_cells_feats_{suffix}.csv"), index=False)

In [510]:
emb_processor.create_cluster_model("kmeans", 
                                   normalize=True,
                                   n_clusters=optimal_nb,
                                   selected_features=selected_feats,
                                   n_init=20,
                                   random_state=1)

In [168]:
emb_processor.save_cluster_model(os.path.join(output_dir, f"cluster_model_cells_feats_{suffix}.pkl"))

In [ ]:
cluster_percentages = [emb_processor.get_cluster_percentages_for_slide(slide_name.split(".")[0]+"", selected_features=selected_feats) for slide_name in df[df["he_slide_id"].str.replace(".svs", "").isin(slide_names)]["he_slide_id"].values]
labels = df[df["he_slide_id"].str.replace(".svs", "").isin(slide_names)].Confirmed_OR.map({1: "Responder", 0: "Non-responder"}).values
labels

In [512]:
df_plot = pd.DataFrame(cluster_percentages)
df_plot["Response to treatment"] = labels
df_plot.iloc[:,:-1] *= 100
df_plot.to_csv(os.path.join(output_dir, f"cluster_percentages_cells_feats_{suffix}.csv"), index=False)
df_plot = df_plot.melt(id_vars='Response to treatment', var_name='Cluster', value_name='Percentage')
df_plot.to_csv(os.path.join(output_dir, f"cluster_percentages_cells_feats_{suffix}_melted.csv"), index=False)

In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Cluster', y='Percentage', hue='Response to treatment', data=df_plot)
ax.set_title('Boxplot of Cluster Percentages Stratified  by Response to Treatment')
plt.savefig(os.path.join(output_dir, f"boxplot_cells_feats_{suffix}.png"), dpi=300, bbox_inches="tight")

# Cell staining distribution analysis

In [13]:
output_dir = "../Results/LUNG/nuclei_IHC/Cell_staining_distribution"
Path(output_dir).mkdir(parents=True, exist_ok=True)

In [ ]:
from sklearn.cluster import KMeans

In [ ]:
labels = df.Confirmed_OR.map({1: "Responder", 0: "Non-responder"}).values
labels

In [16]:
detection_dir = "../Results/LUNG/nuclei_IHC/QuPath/detection_measurements/"
slides = [file.replace(".csv", "") for file in os.listdir(detection_dir)]
detections = [pd.read_csv(os.path.join(detection_dir, file), sep="\t") for file in os.listdir(detection_dir)]

In [ ]:
values = np.concatenate([detections[i]['DAB: Cell: Mean'].values for i in range(len(detections))])
kmeans = KMeans(n_clusters=2, random_state=0).fit(values.reshape(-1, 1))
kmeans.cluster_centers_.mean()

In [18]:
features = {slides[i]: detection[(detection["DAB: Cell: Mean"] > 0.20)&(detection["DAB: Cytoplasm: Min"] > 0.)]["DAB: Cytoplasm: Variance"].mean() for i, detection in enumerate(detections)}

In [19]:
reorganised_features = [features[slide_id.replace(".svs", "")] for slide_id in df["ihc_slide_id"].values]

In [69]:
df_plot = pd.DataFrame({"cytoplasm_dab_avg_var": reorganised_features})
df_plot["Response to treatment"] = labels
df_plot = df_plot.dropna()
df_plot.to_csv(os.path.join(output_dir, "cytoplasm_dab_avg_var.csv"), index=False)

In [ ]:
plt.figure(figsize=(12, 6))
ax = sns.boxplot(x='Response to treatment', y='cytoplasm_dab_avg_var', data=df_plot)
ax.set_title('Boxplot of the average DAB variance measured in cytoplasm stratified  by response to treatment')
pairs=[("Non-responder", "Responder")]
annotator = Annotator(ax, pairs, data=df_plot, x="Response to treatment", y="cytoplasm_dab_avg_var")
annotator.configure(test='Mann-Whitney', text_format='star', loc='inside')
annotator.apply_and_annotate()
plt.savefig(os.path.join(output_dir, "boxplot_cytoplasm_dab_avg_var.png"), dpi=300, bbox_inches="tight")